In [3]:
import cv2
import glob
import numpy as np
import imutils

In [4]:
images = glob.glob('bd_projet_scia/mires/*')

In [38]:
for fname in images[:10]:
        raw_image = cv2.imread(fname)


        scale_percent = 30 # percent of original size
        width = int(raw_image.shape[1] * scale_percent / 100)
        height = int(raw_image.shape[0] * scale_percent / 100)
        dim = (width, height)
        # resize image
        resized = cv2.resize(raw_image, dim, interpolation = cv2.INTER_AREA)
        cv2.imshow('a', resized)
        cv2.waitKey(0)

        gray = cv2.cvtColor(resized,cv2.COLOR_BGR2GRAY)

        bilateral_filtered_image = cv2.bilateralFilter(gray, 5, 175, 175)
        #kernel_size = 3
        #bilateral_filtered_image = cv2.GaussianBlur(gray, (kernel_size, kernel_size), 0)
        cv2.imshow('a', bilateral_filtered_image)
        cv2.waitKey(0)


        low_threshold = 30
        #low_threshold = 50
        high_threshold = 150
        #high_threshold = 175
        edges = cv2.Canny(bilateral_filtered_image, low_threshold, high_threshold, 7)
        cv2.imshow('a', edges)
        cv2.waitKey(0)

        cnts = cv2.findContours(edges.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        cnts = imutils.grab_contours(cnts)
        cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:10]
        screenCnt = None

        # loop over our contours
        if len(cnts) > 0:
                for c in cnts:
                        # approximate the contour
                        peri = cv2.arcLength(c, True)
                        #approx = cv2.approxPolyDP(c, 0.015 * peri, True)
                        approx = cv2.approxPolyDP(c, 0.1 * peri, True)
                
                        # if our approximated contour has four points, then
                        # we can assume that we have found our screen
                        if len(approx) >= 4 and len(approx) <= 12:
                                screenCnt = approx
                                break

                #cv2.drawContours(resized, [screenCnt], -1, (0, 255, 0), 3)
                copy = resized.copy()
                cv2.drawContours(copy, cnts, -1, (0,255,0), 3)
                cv2.imshow("a", copy)
                cv2.waitKey(0)

                for contour in cnts:
                        (x,y,w,h) = cv2.boundingRect(contour)
                        cv2.rectangle(resized, (x,y), (x+w,y+h), (0,255,0), 2)
                cv2.imshow("a", resized)
                cv2.waitKey(0)


        cv2.destroyAllWindows()